# Sentiment Annotation

In [2]:
!pip install --upgrade "numpy>=1.20.3"


  Obtaining dependency information for numpy>=1.20.3 from https://files.pythonhosted.org/packages/11/10/943cfb579f1a02909ff96464c69893b1d25be3731b5d3652c2e0cf1281ea/numpy-1.24.4-cp38-cp38-macosx_10_9_x86_64.whl.metadata
  Using cached numpy-1.24.4-cp38-cp38-macosx_10_9_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp38-cp38-macosx_10_9_x86_64.whl (19.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dial-clean 0.1.1 requires numpy==1.19.5, but you have numpy 1.24.4 which is incompatible.


In [1]:
import openai
import pandas as pd
import re

# Load your dataset from an Excel file
col_names_comment = ["video_link","cid","comment_time","likes","replies","comment_IP","comment"]
col_names_video=["video_id","video_description","video_topic","video_length","video_link","release_date","release_account","likes","num_comments","archives","shares"]

li_comments = pd.read_excel("data/cleaned_li_comments.xlsx")
li_video = pd.read_excel("data/cleaned_li_video.xlsx")
aito_comments = pd.read_excel("data/cleaned_aito_comments.xlsx")
aito_video = pd.read_excel("data/cleaned_aito_video.xlsx")
li_comments.columns = col_names_comment
aito_comments.columns = col_names_comment
li_video.columns = col_names_video
aito_video.columns = col_names_video

In [ ]:

# Check for missing values in each column
missing_values_per_column = li_comments.isnull().any()

print("Columns with missing values:")
print(missing_values_per_column)


Columns with missing values:
video_link      False
cid             False
comment_time    False
likes           False
replies         False
comment_IP      False
comment         False
dtype: bool


In [ ]:

# Check for missing values in each column
missing_values_per_column = aito_comments.isnull().any()

print("Columns with missing values:")
print(missing_values_per_column)

Columns with missing values:
video_link      False
cid             False
comment_time    False
likes           False
replies         False
comment_IP      False
comment         False
dtype: bool


Use english prompt for Chinese comment to test annotation quality

In [ ]:

# Assuming you've already set your API key
openai.api_key = ''

def annotate_article(content):
    system_prompt = "You are a highly knowledgeable assistant asked to analyze Douyin comments."
    user_prompt = (
        f"Analyze this comment:\n{content}\n\n"
        "Strictly just provide the following annotation:\n"
        " What is the sentiment of this comment (0,1,2)\n"
    )
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0125",  # Adjust model as necessary
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )
        annotations = response.choices[0].message['content']
        print(annotations)
        #parts = annotations.split("\n\n")  # Assuming each section is separated by two newlines
        #return parts[:4]  # Return the first four sections
    except Exception as e:
        print(f"An error occurred: {e}")
        return ""



annotate_article("李想在我眼里就是个“司机”")

- Click farm: False
- Sentiment: 1


Use Chinese prompt for Chinese comment to test annotation quality

In [2]:

# Assuming you've already set your API key
openai.api_key = ''

def annotate_comment(content):
    system_prompt = "你是分析抖音评论的专家"
    user_prompt = (
        f"分析这条评论:\n{content}\n"
        "请分析这条评论的情感:(Negative / Neutral / Positive )\n"
        "你的输出应仅为一个单词"
    )

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0125",  # Adjust model as necessary
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )
        annotations = response.choices[0].message['content']
        sentiment = re.sub("[^a-zA-Z]", "", annotations)
        sentiment =sentiment.lower() 
        
       
        return sentiment
        #return truthfulness, sentiment # Return the first four sections
    except Exception as e:
        print(f"An error occurred: {e}")
        return ""

annotate_comment('华为真牛，事故率这么高')


'negative'

# EYEBALL

In [ ]:
num_processed = 0 # Store the number of comments annoted
sentiment = []

In [ ]:
import random

 
# sample 5 comment every batch (batch = 100 here)
def process_comments_in_batches(df,num_processed, batch_size=10, sample_size=5):
    if (len(df)-num_processed)<100:
        batch_size = len(df)-num_processed
        num_processed = len(df)
    results = []
    sentiment = []
    try:
        comments = df.iloc[num_processed:num_processed+batch_size]['comment']
    except:
        comments = df.iloc[num_processed:]['comment']
    
    for comment in comments:
        senti = annotate_comment(comment)
        result = (comment,senti)
        sentiment.append(senti)
        results.append(result)
        
    num_processed += batch_size
    sampled_comments = random.sample(results, min(sample_size, 5)) 
    return results,sampled_comments,sentiment,num_processed



In [ ]:
results, sampled_comments,sentiment_1,num_processed_1= process_comments_in_batches(li_comments,num_processed)

In [ ]:
results

[('渐强扶弱，拜强欺弱，根本区别…再大的博物馆都没用', 'negative'),
 ('英国留学让你更坚定认为自己是英国人', 'neutral'),
 ('老爷子也是一个伟人[抱拳]', 'positive'),
 ('李光耀 要有认祖的初心', 'neutral'),
 ('看不明白吧，这回家多少是华人，确不是中国的铁盟。值得研究。', 'neutral'),
 ('胸怀大志，引领新加坡走向繁荣，功高盖世，敬佩之致。[666][赞]', 'positive'),
 ('汉语在李光耀家族是必须学的', 'neutral'),
 ('明白人', 'neutral'),
 ('为新加坡国父李光耀的中文点赞', 'positive'),
 ('沙发', 'neutral')]

In [ ]:
num_processed_1

100

In [3]:
#aito_comments['sentiment'] = aito_comments['comment'].map(annotate_comment)

An error occurred: Failed to create completion as the model generated invalid Unicode output. Unfortunately, this can happen in rare situations. Consider reviewing your prompt or reducing the temperature of your request. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID req_1af794706a54d21f512620d51606ba66 in your message.) {
  "error": {
    "message": "Failed to create completion as the model generated invalid Unicode output. Unfortunately, this can happen in rare situations. Consider reviewing your prompt or reducing the temperature of your request. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID req_1af794706a54d21f512620d51606ba66 in your message.)",
    "type": "server_error",
    "param": null,
    "code": "invalid_model_output"
  }
}
 500 {'error': {'message': 'Failed to create completion as the model

In [ ]:
li_comments.to_excel("Data/annotated_li_comment.xlsx")

In [4]:
li_comments

,video_link,cid,comment_time,likes,replies,comment_IP,comment
0,https://www.douyin.com/video/6724238509927976206,6724315314135789568,2019-08-13 00:17:49,129,0,无,渐强扶弱，拜强欺弱，根本区别再大的博物馆都没用
1,https://www.douyin.com/video/6724238509927976206,6724307710353139712,2019-08-12 23:48:15,274,2,无,英国留学让你更坚定认为自己是英国人
2,https://www.douyin.com/video/6724238509927976206,6726852199668480000,2019-08-19 20:22:10,107,0,无,老爷子也是一个伟人
3,https://www.douyin.com/video/6724238509927976206,6724309456349589504,2019-08-12 23:55:03,214,2,无,李光耀 要有认祖的初心
4,https://www.douyin.com/video/6724238509927976206,6724324047796400128,2019-08-13 00:51:43,535,3,无,看不明白吧，这回家多少是华人，确不是中国的铁盟。值得研究。
...,...,...,...,...,...,...,...
42335,https://www.douyin.com/video/7350480407307078927,7350522030523710464,2024-03-26 12:19:41,2,0,湖北,理想的
42336,https://www.douyin.com/video/7350480407307078927,7350527617931410432,2024-03-26 12:41:19,2,0,河南,漂亮的豪车
42337,https://www.douyin.com/video/7350480407307078927,7350521168641030144,2024-03-26 12:16:20,6,0,湖南,现在的车越来越理想化了，科技含量高
42338,https://www.douyin.com/video/7350480407307078927,7350523010317200384,2024-03-26 12:23:27,2,0,福建,霸气


In [6]:
aito_comments.to_excel("Data/annotated_aito_comment.xlsx")

In [8]:
li_annotated_comments = pd.read_excel('Data/annotated_li_comment.xlsx')
aito_annotated_comments = pd.read_excel('Data/annotated_aito_comment.xlsx')

In [13]:
li_annotated_comments[li_annotated_comments['sentiment']]

,Unnamed: 0,video_link,cid,comment_time,likes,replies,comment_IP,comment,sentiment
0,0,https://www.douyin.com/video/6724238509927976206,6724315314135789568,2019-08-13 00:17:49,129,0,无,渐强扶弱，拜强欺弱，根本区别再大的博物馆都没用,negative
17,17,https://www.douyin.com/video/6724238509927976206,6724299197681680384,2019-08-12 23:15:13,12,0,无,这老头，也是唯利是图,negative
19,19,https://www.douyin.com/video/6724238509927976206,6724309394584219648,2019-08-12 23:54:48,20,0,无,小心阿三,negative
33,33,https://www.douyin.com/video/6724238509927976206,6724308044043249664,2019-08-12 23:49:33,4,0,无,只有钱才是万能的,negative
35,35,https://www.douyin.com/video/6724238509927976206,6724307449718989824,2019-08-12 23:47:15,13,0,无,别说他了,negative
...,...,...,...,...,...,...,...,...,...
42303,42303,https://www.douyin.com/video/7349840892657732874,7349917184259009536,2024-03-24 21:12:33,1,1,河北,总感觉电动汽车不如燃油汽车,negative
42304,42304,https://www.douyin.com/video/7349840892657732874,7349917011944550400,2024-03-24 21:11:52,1,4,河南,电车不方便,negative
42309,42309,https://www.douyin.com/video/7350192772135095602,7350223065974349824,2024-03-25 16:59:35,6,0,江苏,姐姐完全不拍了？,negative
42330,42330,https://www.douyin.com/video/7350192772135095602,7350336820028680192,2024-03-26 00:21:00,0,0,湖北,你改理想该你不赚钱,negative


In [15]:
li_annotated_comments['sentiment'].value_counts()

sentiment
negative    19145
neutral     14106
positive     9062
Name: count, dtype: int64

In [17]:
aito_annotated_comments['sentiment'].value_counts()

sentiment
negative    14151
neutral     12822
positive     7814
sorry           2
oriented        1
Name: count, dtype: int64